[View in Colaboratory](https://colab.research.google.com/github/aman-chauhan/Image-Coloring/blob/miniCNN/Image_Coloring.ipynb)

# Image-Coloring

***Reference*** - [Let there be Color!: Joint End-to-end Learning of Global and Local Image Priors for Automatic Image Colorization with Simultaneous Classification - Satoshi Iizuka, Edgar Simo-Serra, Hiroshi Ishikawa](http://hi.cs.waseda.ac.jp/~iizuka/projects/colorization/en/)

In [0]:
# !kill -9 -1

## Authenticate

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

## Download data from Google Drive

In [4]:
file_id = '11r-dOQ5Ve2dubFps6D-HTTi7urasqQqn'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
    progress, done = downloader.next_chunk()
    print('.',end='')

downloaded.seek(0)
with open('data.zip','wb') as f:
    f.write(downloaded.read())

......................

In [0]:
!unzip -q data.zip

In [0]:
!rm -r __MACOSX

## Upload weights and epochs

In [0]:
#@title Upload? (1/0)
option = 0 #@param


In [0]:
from google.colab import files

if option==1:
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

## Library Verification

In [9]:
!apt-get -qq install -y graphviz && pip install -q pydot
!pip install imageio

Selecting previously unselected package fontconfig.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.11.94-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.11.94-0ubuntu2) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../01-libjbig0_2.1-3.1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1) ...
Selecting previously unselected package libcdt5.
Preparing to unpack .../02-libcdt5_2.38.0-16ubuntu2_amd64.deb ...
Unpacking libcdt5 (2.38.0-16ubuntu2) ...
Selecting previously unselected package libcgraph6.
Preparing to unpack .../03-libcgraph6_2.38.0-16ubuntu2_amd64.deb ...
Unpacking libcgraph6 (2.38.0-16ubuntu2) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../04-libtiff5_4.0.8-5ubuntu0.1_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.8-5ubuntu0.1) ...
Selecting previously unselected package libwebp6:amd64.
Preparing to unpack .../05-libwebp6_0.6.0-3_amd64.deb ...
Unpa

Setting up libtiff5:amd64 (4.0.8-5ubuntu0.1) ...
Setting up libgraphite2-3:amd64 (1.3.10-2) ...
Setting up libpixman-1-0:amd64 (0.34.0-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up libltdl7:amd64 (2.4.6-2) ...
Setting up libxcb-shm0:amd64 (1.12-1ubuntu1) ...
Setting up libxpm4:amd64 (1:3.5.12-1) ...
Setting up libxt6:amd64 (1:1.1.5-1) ...
Setting up libthai-data (0.1.26-3) ...
Setting up libcdt5 (2.38.0-16ubuntu2) ...
Setting up fontconfig (2.11.94-0ubuntu2) ...
Regenerating fonts cache... done.
Setting up libcgraph6 (2.38.0-16ubuntu2) ...
Setting up libwebp6:amd64 (0.6.0-3) ...
Setting up libcairo2:amd64 (1.14.10-1ubuntu1) ...
Setting up libgvpr2 (2.38.0-16ubuntu2) ...
Setting up libgd3:amd64 (2.2.5-3) ...
Setting up libharfbuzz0b:amd64 (1.4.2-1) ...
Setting up libthai0:amd64 (0.1.26-3) ...
Setting up libxmu6:amd64 (2:1.1.2-2) ...
Setting up libpango-1.0-0:amd64 (1.40.12-1) ...
Setting up libxaw7:amd64 (2:1.0.13-1) ...
Setting up libpangoft2-1.0-0:amd64 (1.4

## Import Libraries

In [10]:
# Keras Libraries
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers.merge import Add
from keras.layers import Input

from keras.utils import to_categorical
from keras.utils import plot_model
from keras.utils import Sequence

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback

from keras import regularizers
from keras import backend as K

# Other libraries
from skimage.color import rgb2lab
from skimage.color import lab2rgb
from imageio import imwrite
from imageio import imread
from PIL import Image
import numpy as np
import json
import os

Using TensorFlow backend.


## CNN Models

### Low-Level Feature Network

In [0]:
def llfn():
    # Input tensor
    llfn_input = Input(batch_shape=(None, None, None, 1), name='llfn_input')

    # Convolutional Layer with 32 3x3 kernels with double stride and same padding
    llfn_conv1 = Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(llfn_input)
    llfn_conv1 = BatchNormalization()(llfn_conv1)

    # Convolutional Layer with 64 3x3 kernels with single stride and same padding
    llfn_conv2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(llfn_conv1)
    llfn_conv2 = BatchNormalization()(llfn_conv2)

    # Convolutional Layer with 64 3x3 kernels with double stride and same padding
    llfn_conv3 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(llfn_conv2)
    llfn_conv3 = BatchNormalization()(llfn_conv3)

    # Convolutional Layer with 128 3x3 kernels with single stride and same padding
    llfn_conv4 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(llfn_conv3)
    llfn_conv4 = BatchNormalization()(llfn_conv4)

    # Convolutional Layer with 128 3x3 kernels with double stride and same padding
    llfn_conv5 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(llfn_conv4)
    llfn_conv5 = BatchNormalization()(llfn_conv5)

    # Convolutional Layer with 256 3x3 kernels with single stride and same padding
    llfn_conv6 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(llfn_conv5)
    llfn_conv6 = BatchNormalization()(llfn_conv6)

    # Model definition
    llfn_model = Model(inputs=llfn_input, outputs=llfn_conv6)

    return llfn_model

### Mid-Level Feature Network

In [0]:
def mlfn():
    # Input tensor
    mlfn_input = Input(batch_shape=(None, None, None, 256), name='mlfn_input')

    # Convolutional Layer with 256 3x3 kernels with single stride and same padding
    mlfn_conv1 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(mlfn_input)
    mlfn_conv1 = BatchNormalization()(mlfn_conv1)

    # Convolutional Layer with 256 3x3 kernels with single stride and same padding
    mlfn_conv2 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu',
                        kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(mlfn_conv1)
    mlfn_conv2 = BatchNormalization()(mlfn_conv2)

    # Model definition
    mlfn_model = Model(inputs=mlfn_input, outputs=mlfn_conv2, name='mlfn_model')

    return mlfn_model

### Global Feature Network

In [0]:
def gfn():
    # Input tensor
    gfn_input = Input(batch_shape=(None, 28, 28, 256), name='gfn_input')

    # Convolutional Layer with 256 3x3 kernels with double stride and same padding
    gfn_conv1 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu',
                       kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(gfn_input)
    gfn_conv1 = BatchNormalization()(gfn_conv1)

    # Convolutional Layer with 256 3x3 kernels with single stride and same padding
    gfn_conv2 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu',
                       kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(gfn_conv1)
    gfn_conv2 = BatchNormalization()(gfn_conv2)

    # Convolutional Layer with 256 3x3 kernels with single stride and same padding
    gfn_conv3 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu',
                       kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(gfn_conv2)
    gfn_conv3 = BatchNormalization()(gfn_conv3)

    # Convolutional Layer with 256 3x3 kernels with single stride and same padding
    gfn_conv4 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu',
                       kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(gfn_conv3)
    gfn_conv4 = BatchNormalization()(gfn_conv4)

    # Flatten the layer
    gfn_flttn = Flatten()(gfn_conv4)

    # Fully Connected Layer with 1024 units
    gfn_fcon1 = Dense(units=1024, activation='relu', kernel_initializer='he_uniform',
                      bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(gfn_flttn)
    gfn_fcon1 = BatchNormalization()(gfn_fcon1)
    gfn_fcon1 = Dropout(0.20)(gfn_fcon1)

    # Fully Connected Layer with 512 units
    gfn_fcon2 = Dense(units=512, activation='relu', kernel_initializer='he_uniform',
                      bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(gfn_fcon1)
    gfn_fcon2 = BatchNormalization()(gfn_fcon2)
    gfn_fcon2 = Dropout(0.20)(gfn_fcon2)

    # Model definition
    gfn_model = Model(inputs=gfn_input, outputs=gfn_fcon2, name='gfn_model')

    return gfn_model

### Classifier Network

In [0]:
def clf():
    # Input tensor
    clf_input = Input(batch_shape=(None, 512), name='clf_input')

    # Fully Connected Layer with 256 units
    clf_fcon1 = Dense(units=256, activation='relu', kernel_initializer='he_uniform',
                      bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(clf_input)
    clf_fcon1 = BatchNormalization()(clf_fcon1)
    clf_fcon1 = Dropout(0.20)(clf_fcon1)

    # Fully Connected Layer with 'output' units
    clf_fcon2 = Dense(units=719, activation='softmax', kernel_initializer='he_uniform',
                      bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(clf_fcon1)

    # Model definition
    clf_model = Model(inputs=clf_input, outputs=clf_fcon2, name='clf_model')

    return clf_model

### Color Network

In [0]:
def color():
    # Input tensor
    color_input = Input(batch_shape=(None, None, None, 256), name='color_input')

    # Convolutional Layer with 128 3x3 kernels with single stride and same padding
    color_conv1 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_input)
    color_conv1 = BatchNormalization()(color_conv1)

    # Convolutional Layer with 64 3x3 kernels with single stride and same padding
    color_conv2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_conv1)
    color_conv2 = BatchNormalization()(color_conv2)

    # Upsampling
    color_upsm1 = UpSampling2D(size=2)(color_conv2)

    # Convolutional Layer with 64 3x3 kernels with single stride and same padding
    color_conv3 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_upsm1)
    color_conv3 = BatchNormalization()(color_conv3)

    # Convolutional Layer with 32 3x3 kernels with single stride and same padding
    color_conv4 = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_conv3)
    color_conv4 = BatchNormalization()(color_conv4)

    # Upsampling
    color_upsm2 = UpSampling2D(size=2)(color_conv4)

    # Convolutional Layer with 32 3x3 kernels with single stride and same padding
    color_conv5 = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_upsm2)
    color_conv5 = BatchNormalization()(color_conv5)

    # Convolutional Layer with 16 3x3 kernels with single stride and same padding
    color_conv6 = Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu',
                         kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_conv5)
    color_conv6 = BatchNormalization()(color_conv6)

    # Upsampling
    color_upsm3 = UpSampling2D(size=2)(color_conv6)

    # Convolutional Layer with 2 3x3 kernels with single stride and same padding
    color_conv7 = Conv2D(filters=2, kernel_size=3, strides=1, padding='same',
                         activation='sigmoid', kernel_initializer='he_uniform', bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(color_upsm3)

    # Model definition
    color_model = Model(inputs=color_input, outputs=color_conv7, name='color_model')

    return color_model

### Full Network

In [0]:
def tile(x, k):
    x = K.expand_dims(x, 1)
    x = K.expand_dims(x, 1)
    x = K.tile(x, [1, k[1], k[2], 1])
    return x


def model():
    color_input = Input(batch_shape=(None, None, None, 1), name='global_color')
    color_branch = llfn()(color_input)
    color_branch = mlfn()(color_branch)

    class_input = Input(batch_shape=(None, 224, 224, 1), name='global_class')
    class_branch = llfn()(class_input)
    class_branch = gfn()(class_branch)

    gfn_units = Dense(units=256, activation='relu', kernel_initializer='he_uniform',
                      bias_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0005))(class_branch)
    gfn_units = BatchNormalization()(gfn_units)

    color_branch = Add()([color_branch, Lambda(tile, arguments={'k': K.shape(color_branch)})(gfn_units)])
    color_branch = color()(color_branch)

    class_branch = clf()(class_branch)

    model = Model(inputs=[color_input, class_input], outputs=[color_branch, class_branch], name='global_model')
    return model

## Data Generator Class

In [0]:
class DataGenerator(Sequence):
    def __init__(self, list_IDs, partition, labels, batch_size=28, n_channel=1, n_classes=719, shuffle=True, augment=False):
        if augment:
            self.list_IDs = list_IDs * 2
        else:
            self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.n_channel = n_channel
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augment = augment
        self.partition = partition
        self.prng = np.random.RandomState(42)
        self.datagen = ImageDataGenerator(rotation_range=45, width_shift_range=0.15, height_shift_range=0.15, shear_range=0.15,
                                          fill_mode='constant', cval=0, zoom_range=0.15, horizontal_flip=True)
        self.on_epoch_end()

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            self.prng.shuffle(self.indexes)

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        return self.__data_generation(list_IDs_temp)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, 224, 224, self.n_channel))
        Y = np.empty((self.batch_size, 224, 224, 2))
        y = np.empty((self.batch_size,), dtype=int)

        for i, ID in enumerate(list_IDs_temp):
            if not self.augment:
                X[i] = np.expand_dims(
                    (imread(os.path.join(os.path.join('data', self.partition), ID)) - 127.5) / 127.5, axis=-1)
                Y[i] = (rgb2lab(imread(os.path.join(os.path.join('data', self.partition + '-target'), ID)))
                        [:, :, 1:] + 128.0) / (255.0)
                y[i] = self.labels[ID]
            else:
                seed = self.prng.randint(0, 1000000)
                X[i] = (self.datagen.random_transform(np.expand_dims(
                    imread(os.path.join(os.path.join('data', self.partition), ID)), axis=-1), seed=seed) - 127.5) / 127.5
                Y[i] = (rgb2lab(self.datagen.random_transform(imread(os.path.join(os.path.join('data', self.partition + '-target'), ID)), seed=seed))
                        [:, :, 1:] + 128.0) / (255.0)
                y[i] = self.labels[ID]
        return ([X, X], [Y, to_categorical(y, num_classes=self.n_classes)])

## Save Callback Class

In [0]:
class SaveCallback(Callback):
    def __init__(self, model):
        self.model_to_save = model

    def on_epoch_end(self, epoch, logs=None):
        self.model_to_save.save_weights('weights.h5')
        d = {}
        if os.path.exists('epochs.json'):
            d = json.load(open('epochs.json'))
        d[epoch] = logs
        json.dump(d, open('epochs.json', 'w'))
        files.download('epochs.json')
        files.download('weights.h5')

## Train Method

In [0]:
def train():
    labels = json.load(open(os.path.join('data', 'labels.json')))
    partition = {'training': None, 'validation': None}
    for x in partition.keys():
        partition[x] = [f for f in os.listdir(os.path.join('data', x)) if os.path.isfile(
            os.path.join(os.path.join('data', x), f))]
        partition[x].sort()
    print('Indices read.')

    n_classes = len({labels[x] for x in labels})
    l = {labels[x] for x in labels}
    l = {x: i for i, x in enumerate(sorted(list(l)))}
    labels = {x: l[labels[x]] for x in labels.keys()}
    json.dump(l, open('mapping.json', 'w'))
    print('Mappings written.')

    training_generator = DataGenerator(partition['training'], 'training', labels, 64, 1, n_classes, True, True)
    validation_generator = DataGenerator(partition['validation'], 'validation', labels, 64, 1, n_classes, True, True)

    fmodel = model()
    if os.path.exists('weights.h5'):
        fmodel.load_weights('weights.h5')

    initial_epoch = 0
    if os.path.exists('epochs.json'):
        initial_epoch = len(json.load(open('epochs.json')).keys())

    cbk = SaveCallback(fmodel)
    fmodel.compile(optimizer='adadelta', loss={
        'color_model': 'mean_squared_error', 'clf_model': 'categorical_crossentropy'}, metrics={'color_model': 'accuracy', 'clf_model': 'accuracy'})
    fmodel.fit_generator(generator=training_generator, epochs=20, verbose=1, callbacks=[
        cbk], validation_data=validation_generator, use_multiprocessing=True, workers=4, initial_epoch=initial_epoch)
    print('Training done.')

In [20]:
train()

Indices read.
Mappings written.
Epoch 1/20
467/561 [=======================>......] - ETA: 3:57 - loss: 10.3425 - color_model_loss: 0.0055 - clf_model_loss: 5.5082 - color_model_acc: 0.5414 - clf_model_acc: 0.1534

561/561 [==============================] - 1561s 3s/step - loss: 9.9908 - color_model_loss: 0.0050 - clf_model_loss: 5.4151 - color_model_acc: 0.5452 - clf_model_acc: 0.1570 - val_loss: 7.8333 - val_color_model_loss: 0.0024 - val_clf_model_loss: 4.7578 - val_color_model_acc: 0.5313 - val_clf_model_acc: 0.1926
Epoch 2/20
169/561 [========>.....................] - ETA: 15:42 - loss: 7.5434 - color_model_loss: 0.0023 - clf_model_loss: 4.7898 - color_model_acc: 0.5804 - clf_model_acc: 0.1930

561/561 [==============================] - 1487s 3s/step - loss: 6.8653 - color_model_loss: 0.0023 - clf_model_loss: 4.6728 - color_model_acc: 0.5983 - clf_model_acc: 0.1989 - val_loss: 5.9869 - val_color_model_loss: 0.0023 - val_clf_model_loss: 4.4458 - val_color_model_acc: 0.5423 - val_clf_model_acc: 0.2207
Epoch 3/20
 67/561 [==>...........................] - ETA: 17:08 - loss: 5.8845 - color_model_loss: 0.0021 - clf_model_loss: 4.3945 - color_model_acc: 0.6196 - clf_model_acc: 0.2222

561/561 [==============================] - 1430s 3s/step - loss: 5.6101 - color_model_loss: 0.0021 - clf_model_loss: 4.3991 - color_model_acc: 0.6134 - clf_model_acc: 0.2245 - val_loss: 5.4426 - val_color_model_loss: 0.0022 - val_clf_model_loss: 4.4719 - val_color_model_acc: 0.5585 - val_clf_model_acc: 0.2122
Epoch 4/20
 17/561 [..............................] - ETA: 20:24 - loss: 5.3481 - color_model_loss: 0.0022 - clf_model_loss: 4.3816 - color_model_acc: 0.6005 - clf_model_acc: 0.2316

Process ForkPoolWorker-40:
Process ForkPoolWorker-29:
Process ForkPoolWorker-31:
Process ForkPoolWorker-37:
Process ForkPoolWorker-38:
Process ForkPoolWorker-32:
Process ForkPoolWorker-39:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-30:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py", line 1035, in rgb2lab
    return xyz2lab(rgb2xyz(rgb), illuminant, observer)
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
KeyboardInterrupt
  File "<ipython-input-17-f96a0e41f473>", line 48, in __data_generation
    Y[i] = (rgb2lab(self.datagen.random_transform(imread(os.path.join(os.path.join('data', self.partition + '-target'), ID)), seed=seed))
  File "/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py", line 264, in apply_transform
    cval=cval) for x_channel in x]
  File "/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py", line 911, in random_transform
    fill_mode=self.fill_mode, cval=self.cval)
  File "/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv

  File "/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py", line 132, in _geometric_transform
    order, mode, cval, extra_arguments, extra_keywords)
  File "/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py", line 1035, in rgb2lab
    return xyz2lab(rgb2xyz(rgb), illuminant, observer)
KeyboardInterrupt
  File "/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py", line 922, in xyz2lab
    arr[mask] = np.power(arr[mask], 1. / 3.)
KeyboardInterrupt


KeyboardInterrupt: ignored

## Download weights and epochs

In [21]:
from google.colab import files
files.download('epochs.json')
files.download('weights.h5')
files.download('mapping.json')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 45002, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil